# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
orders = pd.read_csv("../../Lab Datasets as CSV/Orders.csv")
orders.head(30)
orders

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,541904,581587,22613,2011,12,5,12,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,France,10.20
397920,541905,581587,22899,2011,12,5,12,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,France,12.60
397921,541906,581587,23254,2011,12,5,12,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,France,16.60
397922,541907,581587,23255,2011,12,5,12,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,France,16.60


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
orders_sort = orders.groupby("CustomerID",as_index=False)[["amount_spent"]].sum().sort_values(by="amount_spent",ascending=False)

orders_sort.describe()
print(orders_sort.amount_spent.quantile(0.75))
print(orders_sort.amount_spent.quantile(0.95))
orders_sort

orders_sort["Customer"] = ["VIP Customers" if e >= 1236.54 
                                          else "Preferred Customers" if e >= 471.66
                                          else "-"
                                          for e in orders_sort["amount_spent"]]
orders_sort


1661.6400000000003
5840.181999999983


,CustomerID,amount_spent,Customer
1690,14646,280206.02,VIP Customers
4202,18102,259657.30,VIP Customers
3729,17450,194550.79,VIP Customers
3009,16446,168472.50,VIP Customers
1880,14911,143825.06,VIP Customers
...,...,...,...
4099,17956,12.75,-
3015,16454,6.90,-
1794,14792,6.20,-
3218,16738,3.75,-


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [9]:
orders_nationality = orders.groupby(["CustomerID","Country"],as_index=False)[["amount_spent"]].sum().sort_values(by="amount_spent",ascending=False)
orders_nationality.drop("amount_spent",axis=1,inplace=True)
Orders_Customer = pd.merge(orders_sort,orders_nationality,how="left",on="CustomerID")
#Orders_Customer.groupby(["Country","Customer"],as_index=False)["Customer"].count().sort_values(by="Customer",ascending=False)


In [30]:
Orders_Customer.groupby(["Customer","Country"])[["CustomerID"]].count().sort_values(by="CustomerID",ascending=False)

CustomerID
Customer            Country                   
-                   United Kingdom        1582
VIP Customers       United Kingdom        1228
Preferred Customers United Kingdom        1111
VIP Customers       Germany                 44
                    France                  34
...                                        ...
Preferred Customers Denmark                  1
                    Japan                    1
                    Malta                    1
                    Norway                   1
                    Netherlands              1

[81 rows x 1 columns]

In [34]:
VIP = Orders_Customer[Orders_Customer["Customer"] == "VIP Customers"]
VIP.groupby(["Customer","Country"])[["CustomerID"]].count().sort_values(by="CustomerID",ascending=False)

CustomerID
Customer      Country                       
VIP Customers United Kingdom            1228
              Germany                     44
              France                      34
              Belgium                     15
              Switzerland                 13
              Spain                       10
              Channel Islands              8
              Portugal                     7
              Norway                       7
              Finland                      6
              Austria                      5
              Japan                        5
              Australia                    5
              Italy                        5
              Denmark                      5
              Cyprus                       4
              Netherlands                  3
              Sweden                       3
              EIRE                         3
              Israel                       2
              Poland                       2
              USA                          2
              Malta                        1
              Lithuania                    1
              Lebanon                      1
              Singapore                    1
              European Community           1
              Greece                       1
              Canada                       1
              Iceland                      1

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [42]:
Orders_Customer.drop(Orders_Customer["Customer"] == "-")

KeyError: '[False False False ...  True  True  True] not found in axis'